In [3]:
import os
import numpy as np
import pandas as pd
import time
from scipy.integrate import simpson
from scipy.signal import lombscargle
import fathon
from fathon import fathonUtils as fu
import nolds
import matplotlib.pyplot as plt

In [4]:
# Display a selected file from one of the databases

# List of databases
databases = ['BIDMC-CHF', 'CHF-RR', 'NSR', 'NSR-RR', 'FD']
db_name = databases[2]  # Select a database
n = 1  # Specify which CSV file you want to read (0 for the first file)

# Path to the CSV output directory
csv_output_dir = os.path.join('../data/cleaned_RRIs/', db_name)

# Check if the directory exists
if os.path.exists(csv_output_dir):
    # Get the list of CSV files in the directory
    all_files = sorted(os.listdir(csv_output_dir))
    csv_files = [f for f in all_files if f.endswith('.csv')]

    # Check if there are enough CSV files in the directory to select the nth one
    if len(csv_files) > n:
        selected_file = csv_files[n]  # Select the nth CSV file
        csv_file_path = os.path.join(csv_output_dir, selected_file)

        # Read the selected CSV file into a DataFrame
        df_rris = pd.read_csv(csv_file_path)

        # Display the first rows of the DataFrame
        print(f'Selected file: {selected_file}\n')
        display(df_rris.head(10))
    else:
        print(f"Could not find file. There are fewer than {n} CSV files in the directory {csv_output_dir}.")
else:
    print(f"The directory {csv_output_dir} does not exist.")


Selected file: NSR_16272_RRI.csv



,RR_interval_sec,sample_start,sampling_frequency,record
0,0.984375,73,128,16272
1,0.953125,199,128,16272
2,0.968750,321,128,16272
3,0.953125,445,128,16272
4,0.960938,567,128,16272
5,0.976562,690,128,16272
6,0.984375,815,128,16272
7,0.968750,941,128,16272
8,0.984375,1065,128,16272
9,0.968750,1191,128,16272


In [5]:
## Functions for calculating features

## Time domain features
def calculate_time_domain_features(df_RRIs):
    features = {}

    RRIs = df_RRIs['RR_interval_sec']
    RRIs = RRIs.dropna()  # This removes NaNs, if there are any (just in case)

    mean_RR = RRIs.mean()
    features['mean_RR'] = mean_RR
    
    std_RR = RRIs.std()
    features['std_RR'] = std_RR
    
    min_RR = RRIs.min()
    features['min_RR'] = min_RR
    
    features['media_RR'] = RRIs.median()
    
    features['CV'] = std_RR / mean_RR  # Coefficient of variation
    
    max_RR = RRIs.max()
    features['delta_RRImax'] = max_RR - min_RR

    # SDNN: Standard deviation of NN intervals
    #features['SDNN'] = np.std(RRIs)
    
    # RMSSD: Root mean square of successive differences
    features['RMSSD'] = np.sqrt(np.mean(np.diff(RRIs) ** 2))
    
    # NN50: Number of pairs of successive RR intervals that differ by more than 50 ms = 0.05 seconds
    NN50 = np.sum(np.abs(np.diff(RRIs)) > 0.05)
    features['NN50'] = NN50
    
    # pNN50: Percentage of NN50 among all of the RRIs
    features['pNN50'] = (NN50 / len(RRIs)) * 100

   # NADev: Normalized Absolute Deviation (from the mean)
    NADev = np.mean(np.abs(RRIs - mean_RR)) / mean_RR
    features['NADev'] = NADev
    
    # NADiff: Normalized Absolute Difference
    NADiff = np.mean(np.abs(np.diff(RRIs))) / mean_RR
    features['NADiff'] = NADiff

    features_df = pd.DataFrame([features])  # Wrapping the dictionary in a list creates a single-row DataFrame
    return features_df


## Frequency domain features
# Calculate vLF power, LF power, HF power and total power
def calculate_frequency_features(df_freqs):
    """
    Calculate frequency domain features from power spectral density (PSD) data.

    Parameters:
    df_freqs (DataFrame): A DataFrame containing 'Frequency (Hz)' and 
                          'Power spectral density' columns.

    Returns:
    dict: A dictionary containing total power, VLF, LF, HF power, LF/HF ratio, 
          and LF/HF power in normalized units (n.u.).
    """

    freqs = df_freqs['Frequency (Hz)']
    psd = df_freqs['Power spectral density']

    # Define frequency bands in Hz
    vlf_band = (0.003, 0.04)
    lf_band = (0.04, 0.15)
    hf_band = (0.15, 0.4)

    # Separate the frequencies into the frequency bands
    vlf_freqs = freqs[(freqs >= vlf_band[0]) & (freqs < vlf_band[1])]
    lf_freqs = freqs[(freqs >= lf_band[0]) & (freqs < lf_band[1])]
    hf_freqs = freqs[(freqs >= hf_band[0]) & (freqs <= hf_band[1])]

    # Corresponding power spectral density values
    vlf_psd = psd[(freqs >= vlf_band[0]) & (freqs < vlf_band[1])]
    lf_psd = psd[(freqs >= lf_band[0]) & (freqs < lf_band[1])]
    hf_psd = psd[(freqs >= hf_band[0]) & (freqs <= hf_band[1])]

    # Calculate power in each band using numerical integration
    vlf_power = simpson(vlf_psd, x=vlf_freqs)
    lf_power = simpson(lf_psd, x=lf_freqs)
    hf_power = simpson(hf_psd, x=hf_freqs)

    # Calculate total power in the range 0.003 to 0.4 Hz
    total_power = vlf_power + lf_power + hf_power

    lf_hf_ratio = lf_power/hf_power

    # Normalized units (percentage) for LF and HF power
    lf_power_nu = lf_power/(total_power-vlf_power) * 100
    hf_power_nu = hf_power/(total_power-vlf_power) * 100

    frequency_features = {
        'Total power': total_power,
        'VLF power': vlf_power,
        'LF power': lf_power,
        'HF power': hf_power,
        'LF/HF ratio': lf_hf_ratio,
        'LF power n.u.': lf_power_nu,
        'HF power n.u.': hf_power_nu
    }
    # Convert from dict to dataframe for consistency
    df_frequency_features = pd.DataFrame(frequency_features, index=[0])
    return df_frequency_features


## Nonlinear features
# Sample entropy (SampEn)
def calculate_sample_entropy(rr_intervals):
    # Parameters for SampEn
    m = 2   # Embedding dimension
    r = 0.2 * np.std(rr_intervals)  # Tolerance (20% of the standard deviation)
    
    # Calculate Sample Entropy
    sample_entropy = nolds.sampen(rr_intervals, emb_dim=m, tolerance=r)
    return sample_entropy

# Poincaré plot features (SD1 and SD2):
def calculate_SD1_SD2(rr_intervals):
    # Calculate the differences between consecutive RR intervals
    rr_diff = np.diff(rr_intervals)
    
    # Standard deviation of the RR intervals
    SDRR = np.std(rr_intervals)

    var_SD = np.var(rr_diff) # Variance of successive differences
    # SDSD is the square root of var_SD
    
    # Calculate SD1 and SD2
    SD1 = np.sqrt(var_SD / 2)  # Variability perpendicular to the line y=x (short term variation)
    SD2 = np.sqrt(2 * SDRR**2 - SD1**2)  # Variability along the line y=x (longer term variation)
    
    return SD1, SD2

# Renyi entropy (RyEn)
def calculate_renyi_entropy(rr_intervals, m, alpha):
    """
    Calculate Renyi entropy for RR interval data.

    Parameters:
    - rr_intervals: Array of RR intervals.
    - m: Sequence length for embedding. 
    - alpha: Order of the Renyi entropy.
    - (A possible value pair for m and alpha: m=8, alpha=3)

    Returns:
    - Renyi entropy value.
    """
    if alpha <= 0:
        raise ValueError("Alpha must be > 0")
    if alpha == 1:
        raise ValueError("Renyi entropy undefined for alpha = 1 (use Shannon entropy instead)")

    # Embed data into m-dimensional space
    embedded = np.array([rr_intervals[i:i + m] for i in range(len(rr_intervals) - m + 1)])

    # Estimate probabilities using Gaussian kernel method
    n = embedded.shape[0]
    p = np.zeros(n)
    for i in range(n):
        distances = np.linalg.norm(embedded - embedded[i], axis=1)
        p[i] = np.exp(-distances).sum() - 1  # Exclude self-count
    p /= p.sum()  # Normalize probabilities

    # Calculate Renyi entropy
    renyi_entropy = 1 / (1 - alpha) * np.log2((p ** alpha).sum())
    return renyi_entropy

# Detrended fluctuation analysis (DFA)
def calculate_DFA(rr_intervals, min_window=5, max_window=50, step=5, pol_order=1):
    """
    Calculate DFA (Detrended Fluctuation Analysis), using the Fathon library.

    Parameters:
    - rr_intervals: Array of RR intervals (1D time series)
    - min_window: Minimum window size for DFA
    - max_window: Maximum window size for DFA
    - step: Step size for window sizes
    - pol_order: Order of the polynomial for detrending

    Returns:
    - dfa_alpha: The DFA scaling exponent (Hurst exponent)
    """
    # Convert RR intervals to zero-mean cumulative sum (required by fathon)
    aggregated_rr = fu.toAggregated(rr_intervals)

    # Initialize DFA object
    pydfa = fathon.DFA(aggregated_rr)

    # Define range of window sizes
    win_sizes = fu.linRangeByStep(min_window, max_window, step)

    # Compute fluctuation function
    n, F = pydfa.computeFlucVec(win_sizes, polOrd=pol_order, revSeg=False)

    # Fit fluctuation values to compute the DFA scaling exponent
    dfa_alpha, _ = pydfa.fitFlucVec()
    return dfa_alpha
    

def calculate_nonlinear_features(df_RRIs, test_mode=False):
    """
    Calculate nonlinear features for HRV analysis.
    
    Parameters:
    - df_RRIs: DataFrame containing a column 'RR_interval_sec' with RR intervals in seconds.
    - test_mode: Prints computation times if True.

    Returns:
    - df_nonlinear_features: DataFrame containing nonlinear features.
    """

    # Extract RR intervals
    rr_intervals = df_RRIs['RR_interval_sec']
    rr_intervals = rr_intervals.dropna()  # This removes NaNs, if there are any (just in case)

    # Sample entropy (SampEn)
    start_time = time.time()
    start_time_total = start_time
    sampEn = calculate_sample_entropy(rr_intervals)
    end_time = time.time()
    print(f"Time taken for Sample Entropy calculation: {end_time - start_time:.4f} seconds") if test_mode else None
    
    # Poincaré plot features (SD1 and SD2)
    start_time = time.time()
    SD1, SD2 = calculate_SD1_SD2(rr_intervals)
    end_time = time.time()
    print(f"Time taken for SD1 and SD2 calculation: {end_time - start_time:.4f} seconds") if test_mode else None

    # Renyi entropy (RyEn)
    start_time = time.time()
    m_ryEn = 8  # Sequence length for embedding
    alpha_ryEn = 3  # Order of entropy
    ryEn = calculate_renyi_entropy(rr_intervals, m_ryEn, alpha_ryEn)
    end_time = time.time()
    print(f"Time taken for Renyi Entropy calculation: {end_time - start_time:.4f} seconds") if test_mode else None
    
    # Detrended fluctuation analysis (DFA)
    start_time = time.time()
    dfa_alpha = calculate_DFA(rr_intervals)
    end_time = time.time()
    print(f"Time taken for DFA calculation: {end_time - start_time:.4f} seconds.  DFA_alpha value: {dfa_alpha:.4f}") if test_mode else None

    end_time_total = end_time
    print(f"Time taken for calculation of all nonlinear features: {end_time_total - start_time_total:.4f} seconds\n") if test_mode else None

    nonlinear_features = {
        'SE': sampEn,
        'SD1': SD1,
        'SD2': SD2,
        'RE': ryEn,
        'DFA': dfa_alpha
    }
    
    # Convert from dict to dataframe for consistency
    df_nonlinear_features = pd.DataFrame(nonlinear_features, index=[0])
    return df_nonlinear_features

In [47]:
# Calculate features

# Choose segment length
N = 500
print(f'Segment length: N = {N}\n')

# Choose which features to calculate, among time-, frequency- and nonlinear features
#features_to_calculate = 'time' # time domain features
#features_to_calculate = 'frequency' # frequency domain features
features_to_calculate = 'nonlinear' # nonlinear features

# Test mode = True: Run code for only two databases (one CHF, one NSR), and only 3 subjects from each of them.
test_mode = False
if test_mode:
    n_subjects = 2 # Maximum number of subjects per database in test mode
    n_segments_per_subject = 4 # Maximum number of segments analyzed per subject in test mode

# Base directory where the CSV files are stored
base_dir = '../data/cleaned_RRIs'

# Directory where the calculated features CSV files are to be saved
calculated_features_dir = '../data/calculated_features'
os.makedirs(calculated_features_dir, exist_ok=True)

# List of subdirectories representing each database
databases = ['BIDMC-CHF', 'CHF-RR', 'NSR', 'NSR-RR', 'FD']
databases_chf = ['BIDMC-CHF', 'CHF-RR'] # CHF = 1
databases_nsr = ['NSR', 'NSR-RR', 'FD'] # CHF = 0

if test_mode:
    databases = [databases_chf[0], databases_nsr[0]]

# Define file paths for saving calculated feature values
if features_to_calculate == 'time':
    # Define file name for saving calculated feature values
    filename_calculated_features_csv = f'N{N}_time_features.csv'
    # File names for means and standard deviations of calculated feature values
    filename_calculated_features_means = f'N{N}_time_features_means.csv'
    filename_calculated_features_SDs = f'N{N}_time_features_SDs.csv'
    
    # Define headers for the CSV file
    headers = ['subject_id', 'database', 'CHF', 'segment_number', 'mean_RR', 'std_RR', 
                'min_RR', 'media_RR', 'CV', 'delta_RRImax', 'RMSSD', 'NN50', 'pNN50', 'NADev', 'NADiff']

elif features_to_calculate == 'frequency':
    filename_calculated_features_csv = f'N{N}_frequency_features.csv'
    filename_calculated_features_means = f'N{N}_frequency_features_means.csv'
    filename_calculated_features_SDs = f'N{N}_frequency_features_SDs.csv'

    headers = ['subject_id', 'database', 'CHF', 'segment_number', 'Total power', 'VLF power', 
                'LF power', 'HF power', 'LF/HF ratio', 'LF power n.u.', 'HF power n.u.']

elif features_to_calculate == 'nonlinear':
    filename_calculated_features_csv = f'N{N}_nonlinear_features.csv'
    filename_calculated_features_means = f'N{N}_nonlinear_features_means.csv'
    filename_calculated_features_SDs = f'N{N}_nonlinear_features_SDs.csv'
    
    headers = ['subject_id', 'database', 'CHF', 'segment_number', 'SE', 'SD1', 'SD2', 'RE', 'DFA']

# Full file path to where the calculated feature values are to be saved
path_calculated_features_csv = os.path.join(calculated_features_dir, filename_calculated_features_csv)

# Create an empty DataFrame with the relevant headers and save to CSV
pd.DataFrame(columns=headers).to_csv(path_calculated_features_csv, index=False)

# Measure the time it takes to calculate the features (all subjects)
start_time_all = time.time()

# Iterate through each database
for db in databases:
    db_path = os.path.join(base_dir, db)
    
    # Check if the directory exists
    if os.path.exists(db_path):
        subjects_count = 0 if test_mode else None
        
        for file in os.listdir(db_path):
            # Measure the time it takes to calculate the features of one subject
            start_time_subj = time.time()
            
            if file.endswith(".csv"):
                if test_mode:
                    subjects_count += 1
                    if subjects_count > n_subjects:
                        break
                    
                # Load each CSV file
                file_path = os.path.join(db_path, file)
                df_rris = pd.read_csv(file_path) # RRI data of one subject
                subject_code_name = df_rris['record'].iloc[0] # Retrieve the record (subject code name)
                subject_code_name = str(subject_code_name) # Convert to a string to ensure it is treated as text
                
                # Partition the data into intervals of 500 RRIs
                num_segments = len(df_rris) // N

                # Print the database and subject file name to track progress
                if test_mode:
                    print(f"\nProcessing database: {db}, subject: {file}.\tn_segments_per_subject (test mode): {n_segments_per_subject}.")
                else:
                    print(f"\nProcessing database: {db}, subject: {file}.\tNumber of segments: {num_segments}.")
                
                # Process complete segments
                for i in range(num_segments):
                    if test_mode:
                        #segments_count += 1
                        if i >= n_segments_per_subject-1:
                            break

                    # Current RRI segment
                    df_segment = df_rris.iloc[i*N:(i+1)*N]
                    
                    # Convert the identifying information into a DataFrame
                    id_info = {
                        'subject_id': subject_code_name,
                        'database': db,
                        'CHF': 1 if 'CHF' in db else 0,
                        'segment_number': i + 1
                    }
                    df_id_info = pd.DataFrame([id_info])  # Wrap in a list to make it a single-row DataFrame

                    if features_to_calculate == 'time':
                        # Time domain features
                        df_calculated_features = calculate_time_domain_features(df_segment)
                        
                    elif features_to_calculate == 'frequency':
                        # Frequency domain features
                        df_freqs = transform_to_frequency_domain(df_segment)
                        df_calculated_features = calculate_frequency_features(df_freqs)

                    elif features_to_calculate == 'nonlinear':
                        df_calculated_features = calculate_nonlinear_features(df_segment, test_mode=test_mode)
                        
                    # Concatenate the identifying information and features
                    combined_df = pd.concat([df_id_info, df_calculated_features], axis=1)

                    # Append to CSV
                    combined_df.to_csv(path_calculated_features_csv, mode='a', header=False, index=False)

                # Handle the last segment if it's not complete
                remaining_samples = len(df_rris) % N
                if remaining_samples >= N // 2:
                    # Last segment
                    # Overlap with the previous segment to ensure we have 500 samples
                    df_segment = df_rris.iloc[-N:]
                    
                    # Convert the identifying information into a DataFrame
                    id_info = {
                        'subject_id': subject_code_name,
                        'database': db,
                        'CHF': 1 if 'CHF' in db else 0,
                        'segment_number': num_segments + 1
                    }
                    df_id_info = pd.DataFrame([id_info])  # Wrap in a list to make it a single-row DataFrame

                    if features_to_calculate == 'time':
                        # Time domain features
                        df_calculated_features = calculate_time_domain_features(df_segment)
                        
                    elif features_to_calculate == 'frequency':
                        # Frequency domain features
                        df_freqs = transform_to_frequency_domain(df_rris)
                        df_calculated_features = calculate_frequency_features(df_freqs)
                        #print(f'type(df_freq_features): {type(df_freq_features)}')

                    elif features_to_calculate == 'nonlinear':
                        df_calculated_features = calculate_nonlinear_features(df_segment)

                    # Concatenate the identifying information and features
                    combined_df = pd.concat([df_id_info, df_calculated_features], axis=1)

                    # Append to CSV
                    combined_df.to_csv(path_calculated_features_csv, mode='a', header=False, index=False)
            end_time_subj = time.time()
            duration_subj = end_time_subj - start_time_subj
            print(f"Time taken to calculate features for the subject: {duration_subj:.2f} seconds")
            
# End timing
end_time_all = time.time()
# Calculate and print the duration
duration_all = end_time_all - start_time_all
print(f"Time taken to calculate features: {duration_all:.2f} seconds")

Segment length: N = 500


Processing database: BIDMC-CHF, subject: BIDMC-CHF_chf01_RRI.csv.	Number of segments: 149.
Time taken to calculate features for the subject: 26.24 seconds

Processing database: BIDMC-CHF, subject: BIDMC-CHF_chf02_RRI.csv.	Number of segments: 181.
Time taken to calculate features for the subject: 31.51 seconds

Processing database: BIDMC-CHF, subject: BIDMC-CHF_chf03_RRI.csv.	Number of segments: 156.
Time taken to calculate features for the subject: 26.75 seconds

Processing database: BIDMC-CHF, subject: BIDMC-CHF_chf04_RRI.csv.	Number of segments: 221.
Time taken to calculate features for the subject: 36.11 seconds

Processing database: BIDMC-CHF, subject: BIDMC-CHF_chf05_RRI.csv.	Number of segments: 236.
Time taken to calculate features for the subject: 25.52 seconds

Processing database: BIDMC-CHF, subject: BIDMC-CHF_chf06_RRI.csv.	Number of segments: 223.
Time taken to calculate features for the subject: 23.08 seconds

Processing database: BIDMC-CHF, subjec

NameError: name 'start_time_all' is not defined

In [48]:
# Calculate means and standard deviations of the calculated features and save to CSV

# Load the features CSV into a DataFrame
df_segments_features = pd.read_csv(path_calculated_features_csv)

# Exclude metadata columns
columns_to_exclude = ['subject_id', 'database', 'segment_number']
segments_features_data = df_segments_features.drop(columns=columns_to_exclude)

# Group segments by CHF status
segments_features_grouped_by_CHF = segments_features_data.groupby(df_segments_features['CHF'])

# Get the mean values
mean_values = segments_features_grouped_by_CHF.mean()

# Get the standard deviations
std_values = segments_features_grouped_by_CHF.std()

# Drop the duplicate CHF column (keeping the grouping column, which indicates CHF = 0 or 1)
mean_values = mean_values.drop(columns=['CHF'])
std_values = std_values.drop(columns=['CHF'])

# Save the mean and standard deviation values to separate CSV files
path_calculated_features_means = os.path.join(calculated_features_dir, filename_calculated_features_means)
path_calculated_features_SDs = os.path.join(calculated_features_dir, filename_calculated_features_SDs)
mean_values.to_csv(path_calculated_features_means)
std_values.to_csv(path_calculated_features_SDs)

# Display the calculated means and standard deviations
print("\nMean values:")
display(mean_values)
print("\nStandard deviation values:")
display(std_values)


Mean values:


,SE,SD1,SD2,RE,DFA
CHF,,,,,
0,1.207861,0.030471,0.078922,8.949994,1.041602
1,0.702377,0.079304,0.093554,8.929786,0.750057



Standard deviation values:


,SE,SD1,SD2,RE,DFA
CHF,,,,,
0,0.482457,0.030608,0.043245,0.014165,0.221446
1,0.545744,0.072930,0.071990,0.038129,0.282906


In [58]:
# For the specified segment length, *combine the three CSVs* (time domain features, frequency domain features and nonlinear features)
# into one CSV with all features (means and standard deviations of the feature values).

# Segment length
N = 500

calculated_features_dir = '../data/calculated_features'  # Directory where the CSV files are stored

# File paths to the CSVs with means and standard deviations of the features
# Means
filename_time_features_means = os.path.join(calculated_features_dir, f'N{N}_time_features_means.csv')
filename_frequency_features_means = os.path.join(calculated_features_dir, f'N{N}_frequency_features_means.csv')
filename_nonlinear_features_means = os.path.join(calculated_features_dir, f'N{N}_nonlinear_features_means.csv')
# Standard Deviations
filename_time_features_SDs = os.path.join(calculated_features_dir, f'N{N}_time_features_SDs.csv')
filename_frequency_features_SDs = os.path.join(calculated_features_dir, f'N{N}_frequency_features_SDs.csv')
filename_nonlinear_features_SDs = os.path.join(calculated_features_dir, f'N{N}_nonlinear_features_SDs.csv')

# Load the CSV files into DataFrames
# Means
df_time_features_means = pd.read_csv(filename_time_features_means)
df_frequency_features_means = pd.read_csv(filename_frequency_features_means)
df_nonlinear_features_means = pd.read_csv(filename_nonlinear_features_means)
# Standard deviations
df_time_features_SDs = pd.read_csv(filename_time_features_SDs)
df_frequency_features_SDs = pd.read_csv(filename_frequency_features_SDs)
df_nonlinear_features_SDs = pd.read_csv(filename_nonlinear_features_SDs)


# Merge the Dataframes on 'CHF'.
# Means
df_all_features_means = df_time_features_means.merge(
    df_frequency_features_means, on='CHF').merge(
    df_nonlinear_features_means, on='CHF')

# Standard deviations
df_all_features_SDs = df_time_features_SDs.merge(
    df_frequency_features_SDs, on='CHF').merge(
    df_nonlinear_features_SDs, on='CHF')


# File path for saving the combined features
path_all_features_means = os.path.join(calculated_features_dir, f'N{N}_all_features_means.csv')
path_all_features_SDs = os.path.join(calculated_features_dir, f'N{N}_all_features_SDs.csv')

# Save the combined DataFrame to a new CSV file
df_all_features_means.to_csv(path_all_features_means, index=False)
df_all_features_SDs.to_csv(path_all_features_SDs, index=False)

print(f"Combined mean values saved to: {path_all_features_means}")
print(f"Combined standard deviations saved to: {path_all_features_SDs}")

Combined mean values saved to: ../data/calculated_features\N500_all_features_means.csv
Combined standard deviations saved to: ../data/calculated_features\N500_all_features_SDs.csv


In [6]:
# Create a summary DataFrame similar to Wang et al.

calculated_features_dir = '../data/calculated_features'  # Directory where the CSV files are stored
N=500
path_all_features_means = os.path.join(calculated_features_dir, f'N{N}_all_features_means.csv')
path_all_features_SDs = os.path.join(calculated_features_dir, f'N{N}_all_features_SDs.csv')

#style = 1 # More straightforward code for presenting the results
style = 2 # More similar to the table in Wang et al. 

# Import CSVs of calculated means and standard deviations
mean_values = pd.read_csv(path_all_features_means) # e.g. ../data/calculated_features/N500_all_features_means.csv
std_values = pd.read_csv(path_all_features_SDs)

if style == 1:
    summary_df = pd.DataFrame()
    # Add mean and std for NSR and CHF groups
    for column in mean_values.columns:
        nsr_mean = mean_values.loc[0, column]
        nsr_std = std_values.loc[0, column]
        chf_mean = mean_values.loc[1, column]
        chf_std = std_values.loc[1, column]
    
        summary_df[column] = [f"{nsr_mean:.2f}±{nsr_std:.2f}", f"{chf_mean:.2f}±{chf_std:.2f}"]
    
    # Set the index to specify which rows are for NSR and CHF
    summary_df.index = ['NSR', 'CHF']
    
    print("\nSummary DataFrame with means ± standard deviations:")
    display(summary_df)

elif style == 2:
    # Create lists for features, NSR, CHF, and p-value placeholders
    hrv_indices = list(mean_values.columns)
    nsr_values = []
    chf_values = []
    p_values = ["-" for _ in range(len(hrv_indices))]  # Placeholder for p-values, can be calculated separately
    
    # Populate the NSR and CHF columns with mean ± std values
    for column in hrv_indices:
        nsr_mean = mean_values.loc[0, column]
        nsr_std = std_values.loc[0, column]
        chf_mean = mean_values.loc[1, column]
        chf_std = std_values.loc[1, column]

        if column == 'CHF': # For this row in the printed table, show only 1 or 0 for simplicity.
            nsr_values.append(f"{nsr_mean}")
            chf_values.append(f"{chf_mean}")
        else:
            nsr_values.append(f"{nsr_mean:.2f}±{nsr_std:.2f}")
            chf_values.append(f"{chf_mean:.2f}±{chf_std:.2f}")
    
    # Create a new DataFrame with columns similar to Wang et al. table
    summary_df = pd.DataFrame({
        "HRV indices": hrv_indices,
        "NSR": nsr_values,
        "CHF": chf_values,
        "p-Value": p_values  # Placeholder for now
    })
    
    # Display the updated summary DataFrame
    print("\nSummary DataFrame with means ± standard deviations:")
    display(summary_df)


Summary DataFrame with means ± standard deviations:


,HRV indices,NSR,CHF,p-Value
0,CHF,0,1,-
1,mean_RR,0.79±0.15,0.69±0.12,-
2,std_RR,0.06±0.04,0.09±0.07,-
3,min_RR,0.65±0.12,0.60±0.10,-
4,media_RR,0.79±0.16,0.68±0.12,-
5,CV,0.08±0.04,0.12±0.09,-
6,delta_RRImax,0.46±0.32,0.73±0.39,-
7,RMSSD,0.04±0.04,0.11±0.10,-
8,NN50,42.96±63.47,38.99±64.77,-
9,pNN50,8.59±12.69,7.80±12.95,-


In [18]:
# Script for copying the calculated features in \data\calculated_features into the 
# sub-folder \data\calculated_features\N500_all_features.
# This was added at a later stage in the project, to prepare the inputs in csvs organized by database 
# and subject (one csv for each subject), with each row in a subject's csv corresponding to one RRI segment.

# Directory with previously calculated features
features_dir_load = os.path.join("..", "data", "calculated_features")

# Files to combine
feature_files = ['N500_time_features.csv', 'N500_frequency_features.csv', 'N500_nonlinear_features.csv']

dfs = []

for file in feature_files:
    file_path = os.path.join(features_dir_load, file)
    df = pd.read_csv(file_path)
    dfs.append(df)

df1, df2, df3 = dfs[0], dfs[1], dfs[2]

databases = df1['database'].unique() # Assuming 'database' column is consistent across all three dfs
print("Databases found:", databases)

# Directory to save the updated RRI data with DL features
features_dir_save = os.path.join("..", "data", "calculated_features", "N500_all_features")

# N500_all_features, file name example: BIDMC-CHF_chf01_RRI (f'{db}_{subject_id}_features')
# Columns: segment_id, database, label, {feature names}

for db_name in databases:
    print(f"\nProcessing database: {db_name}")

    # Organize subjects by database
    db_features_dir_save = os.path.join(features_dir_save, db_name)
    os.makedirs(db_features_dir_save, exist_ok=True)

    # Filter DataFrames for the current database
    df1_db = df1[df1['database'] == db_name]
    df2_db = df2[df2['database'] == db_name]
    df3_db = df3[df3['database'] == db_name]

    # Get Unique Subjects for the current database
    subjects = df1_db['subject_id'].unique() # Assuming 'subject_id' is consistent in all dfs
    print(f"  Subjects in {db_name}: {subjects}")

    # Iterate through each subject in the current database
    for subject_id in subjects:
        print(f"    Processing subject: {subject_id}")

        # Filter DataFrames for the current subject
        subject_df1 = df1_db[df1_db['subject_id'] == subject_id]
        subject_df2 = df2_db[df2_db['subject_id'] == subject_id]
        subject_df3 = df3_db[df3_db['subject_id'] == subject_id]

        # Merge the DataFrames for the current subject
        # Use 'merge' based on the common columns. 'inner' join to keep only matching rows.
        merged_df = pd.merge(subject_df1, subject_df2, on=['subject_id', 'database', 'CHF', 'segment_number'], how='inner')
        merged_df = pd.merge(merged_df, subject_df3, on=['subject_id', 'database', 'CHF', 'segment_number'], how='inner')

        # Rename 'CHF' to 'label'
        merged_df = merged_df.rename(columns={'CHF': 'label'})
        
        # Replace the 'segment_number' column with the column 'segment_id' which will contain some more information
        num_segments = len(merged_df)
        merged_df['segment_number'] = merged_df['segment_number'].apply(
            lambda segment_no: f"{subject_id}_{str(segment_no).zfill(len(str(num_segments)))}"
        )
        merged_df = merged_df.rename(columns={'segment_number': 'segment_id'})

        # Drop duplicate 'segment_id's, if any
        merged_df = merged_df.drop_duplicates(subset=['segment_id'], keep='first')
        
        # Save merged Dataframe to csv
        output_file = f'{db_name}_{subject_id}_features.csv'
        save_file_path = os.path.join(db_features_dir_save, output_file)
        merged_df.to_csv(save_file_path, index=False)


Databases found: ['BIDMC-CHF' 'CHF-RR' 'NSR' 'NSR-RR' 'FD']

Processing database: BIDMC-CHF
  Subjects in BIDMC-CHF: ['chf01' 'chf02' 'chf03' 'chf04' 'chf05' 'chf06' 'chf07' 'chf08' 'chf09'
 'chf10' 'chf11' 'chf12' 'chf13' 'chf14' 'chf15']
    Processing subject: chf01
    Processing subject: chf02
    Processing subject: chf03
    Processing subject: chf04
    Processing subject: chf05
    Processing subject: chf06
    Processing subject: chf07
    Processing subject: chf08
    Processing subject: chf09
    Processing subject: chf10
    Processing subject: chf11
    Processing subject: chf12
    Processing subject: chf13
    Processing subject: chf14
    Processing subject: chf15

Processing database: CHF-RR
  Subjects in CHF-RR: ['chf201' 'chf202' 'chf203' 'chf204' 'chf205' 'chf206' 'chf207' 'chf208'
 'chf209' 'chf210' 'chf211' 'chf212' 'chf213' 'chf214' 'chf215' 'chf216'
 'chf217' 'chf218' 'chf219' 'chf220' 'chf221' 'chf222' 'chf223' 'chf224'
 'chf225' 'chf226' 'chf227' 'chf228' 'ch